In [15]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

# Set CPU as available physical device
tf.config.set_visible_devices([], 'GPU')

from pymongo import MongoClient
from auth import username, password
import urllib
from pprint import pprint


In [2]:
client = MongoClient('mongodb://' + urllib.parse.quote_plus(username) + ':' + urllib.parse.quote_plus(password) + '@198.211.115.252')

In [3]:
gs_folder_bert = "C:/Users/Ryan/Desktop/projects/ConspiracyBERT/config"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'BERT_LARGE_CASED',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'bert_model.ckpt.meta',
 'vocab.txt',
 'wwm_uncased_L-24_H-1024_A-16']

In [4]:
# Set up tokenizer to generate Tensorflow dataset
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
     do_lower_case=True)

# print("Vocab size:", len(tokenizer.vocab))

# tokens = tokenizer.tokenize("Hello TensorFlow!")
# print(tokens)
# ids = tokenizer.convert_tokens_to_ids(tokens)
# print(ids)

# tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

In [5]:
data = {}

conspiracies = ['chemtrails']
for conspiracy in conspiracies:

    chemcol = client[conspiracy]['hashtag_chemtrails'] #todo
    cursor = chemcol.find({})
    for document in cursor:
        try:
            data[conspiracy].append(document)
        except:
            data.update({conspiracy : [document]})

In [22]:
def encode_sentence(s, tokenizer):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [27]:
encoded_sentances = np.array([np.array(encode_sentence(row['text'], tokenizer)) for row in data[conspiracy] for conspiracy in conspiracies])

# for conspiracy in conspiracies:
#     conspiracy_tweets = data[conspiracy]
#     for row in conspiracy_tweets: 
#         tokens = tokenizer.tokenize(row['text'])
# #         print(tokens)
#         ids = tokenizer.convert_tokens_to_ids(tokens)
#         print(ids)

In [33]:
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out

In [45]:
padded_encoded_sentances = numpy_fillna(encoded_sentances)
input_mask = np.ones_like(padded_encoded_sentances)
inputs_types = np.zeros(padded_encoded_sentances.shape)

In [40]:
padded_encoded_sentances.shape

(75118, 449)

In [46]:
inputs_types.shape

(75118, 449)